<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/Xception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install wandb -qU
!wandb login

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(123)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/wear particles'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)
num_classes = len(class_names)

Found 740 files belonging to 5 classes.
Using 592 files for training.
Found 740 files belonging to 5 classes.
Using 148 files for validation.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [6]:
xception = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None)
data_augmentation = tf.keras.Sequential([layers.RandomFlip("horizontal_and_vertical"),layers.RandomRotation(0.2),])
model_xception=Sequential([
  #data_augmentation,
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  xception,
  layers.Flatten(),
  layers.Dense(32, activation='relu'),
  #layers.Dropout(0.5), 
  layers.Dense(num_classes, activation='softmax')
])
xception.trainable=False
model_xception.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 xception (Functional)       (None, None, None, 2048)  20861480  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 32)                3211296   
                                                                 
 dense_1 (Dense)             (None, 5)                 165       
                                                                 
Total params: 24,072,941
Trainable params: 3,211,461
Non-trainable params: 20,861,480
_________________________________________________________________


In [7]:
model_xception.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [8]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_learning", 
      name = "Xception_final",
      notes = "flatten ?"
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [9]:
%%time
epochs=40
history = model_xception.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/40
19/19 [==============================] - ETA: 0s - loss: 0.9153 - accuracy: 0.8209

wandb: Adding directory to artifact (/content/wandb/run-20221018_075901-2q6ayjw9/files/model-best)... Done. 1.0s


19/19 [==============================] - 199s 6s/step - loss: 0.9153 - accuracy: 0.8209 - val_loss: 0.2296 - val_accuracy: 0.9595
Epoch 2/40
19/19 [==============================] - ETA: 0s - loss: 0.0519 - accuracy: 0.9899

wandb: Adding directory to artifact (/content/wandb/run-20221018_075901-2q6ayjw9/files/model-best)... Done. 1.0s


19/19 [==============================] - 22s 1s/step - loss: 0.0519 - accuracy: 0.9899 - val_loss: 0.1157 - val_accuracy: 0.9797
Epoch 3/40
19/19 [==============================] - 4s 184ms/step - loss: 0.0068 - accuracy: 0.9983 - val_loss: 0.1549 - val_accuracy: 0.9797
Epoch 4/40
19/19 [==============================] - 4s 184ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.1668 - val_accuracy: 0.9730
Epoch 5/40
19/19 [==============================] - 4s 186ms/step - loss: 2.8748e-04 - accuracy: 1.0000 - val_loss: 0.1504 - val_accuracy: 0.9797
Epoch 6/40
19/19 [==============================] - 4s 185ms/step - loss: 1.8465e-04 - accuracy: 1.0000 - val_loss: 0.1442 - val_accuracy: 0.9865
Epoch 7/40
19/19 [==============================] - 4s 185ms/step - loss: 1.4852e-04 - accuracy: 1.0000 - val_loss: 0.1423 - val_accuracy: 0.9865
Epoch 8/40
19/19 [==============================] - 4s 187ms/step - loss: 1.3042e-04 - accuracy: 1.0000 - val_loss: 0.1407 - val_accuracy: 0.9865
Epo

In [10]:
wandb.finish()

accuracy,▁███████████████████████████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▆▆▅▆███████████████████████████████████
val_loss,█▁▃▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
accuracy,1.0
best_epoch,1
best_val_loss,0.11572
epoch,39
loss,3e-05
val_accuracy,0.98649
